In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

In [0]:
!wget https://baidu-nlp.bj.bcebos.com/ERNIE_stable-1.0.1.tar.gz

In [0]:
!tar -xzvf ERNIE_stable-1.0.1.tar.gz

In [0]:
!unzip -o chinese_L-12_H-768_A-12.zip

In [0]:
!7z x '/content/drive/My Drive/baidu_ernie.7z' -r -o/content/baidu_ernie

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [0]:
import jieba
jieba.load_userdict("/content/userdict.txt")
with open('/content/stopwords.txt') as f:
    stopWords = [line.strip() for line in f.readlines()]

In [0]:
import pandas as pd
from tokenization import BasicTokenizer

tokenizer = BasicTokenizer()

df = pd.read_csv('/content/bert_remove.csv')
# 进行分词处理
df['cutted'] = df['comment'].apply(lambda x: tokenizer.tokenize(x))
#df['cutted'] = df['comment'].apply(lambda x: jieba.lcut(x))

# 准备训练测试数据集
train_x = list(df['cutted'][:int(len(df)*0.7)])
train_y = list(df['label'][:int(len(df)*0.7)])

valid_x = list(df['cutted'][int(len(df)*0.7):int(len(df)*0.85)])#
valid_y = list(df['label'][int(len(df)*0.7):int(len(df)*0.85)])#int(len(df)*0.85)

test_x = list(df['cutted'][int(len(df)*0.85):])
test_y = list(df['label'][int(len(df)*0.85):])

In [0]:
print(train_x[24:25])

[['送', '了', '1', '个', '小', '时', '，', '到', '了', '东', '西', '都', '冷', '了']]


In [0]:
!pip install kashgari-tf

In [0]:
import kashgari
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.tasks.classification import CNN_Model
from kashgari.tasks.classification import KMax_CNN_Model
kashgari.config.use_cudnn_cell = True

BERT_PATH = '/content/baidu_ernie'

# 使用 embedding 初始化模型
#model = CNN_Model(embed)
hyper = KMax_CNN_Model.get_default_hyper_parameters()
print(hyper)
#hyper['layer_bi_lstm']['units'] = 32


╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

{'spatial_dropout': {'rate': 0.2}, 'conv_0': {'filters': 180, 'kernel_size': 1, 'kernel_initializer': 'normal', 'padding': 'valid', 'activation': 'relu'}, 'conv_1': {'filters': 180, 'kernel_size': 2, 'kernel_initializer': 'normal', 'padding': 'valid', 'activation': 'relu'}, 'conv_2': {'filters': 180, 'kernel_size': 3, 'kernel_initializer': 'normal', 'padding': 'valid', 'activation': 'relu'}, 'conv_3': {'filters': 180, 'kernel_size': 4, 'kernel_initializer': 'normal', 'padding': 'valid', 'activation': 'relu'}, 'maxpool_i4': {'k': 3}, 'merged_tensor': {'axis': 1}, 'dropout': {'rate': 0.6}, 'dense': {'units': 144, 'activation': 'relu'}, 'activation_layer': {'activation': 'softmax'}}


In [0]:
hyper['conv_1']['kernel_size'] = 3

In [0]:
# 初始化 Embedding
embed = BERTEmbedding(BERT_PATH,
                     task=kashgari.CLASSIFICATION,
                     sequence_length=128)

model = KMax_CNN_Model(embed,hyper_parameters=hyper)
# 先只训练一轮
model.fit(train_x, train_y, valid_x, valid_y, batch_size=32, epochs=10)


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 13824000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
model.evaluate(test_x, test_y, batch_size=512)

              precision    recall  f1-score   support

           0     0.9849    0.9865    0.9857      1256
           1     0.9298    0.9221    0.9259       244

    accuracy                         0.9760      1500
   macro avg     0.9573    0.9543    0.9558      1500
weighted avg     0.9759    0.9760    0.9760      1500



In [0]:
model.save('bert_model')

In [0]:
import kashgari
kashgari.utils.convert_to_saved_model(model, 'tf_bert_model', version=1)

In [0]:
test = pd.read_csv("test_bert_remove.csv")
print(test.head())

                                     id                                            comment
0  0011f384-9e54-4fb4-a272-330a6cab6804       糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试
1  00223e4f-47e1-4fc8-9657-06444a7de9a5  满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...
2  00225350-c169-435c-84cf-970068df5b12                              好喝！经常会再去买来喝！就是排队的人太多了
3  00a3190c-90c1-44c3-b809-7a9b1314cd27                              三个人订的四人餐，菜量大没吃完，问道不错。
4  00b3f76e-fda3-42cd-8884-25e03a5dba64  好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！喜欢所有菜和肉，两女一男吃两份两人餐没吃...


In [0]:
import numpy as np
predict_list = list(test['comment'].apply(lambda x: tokenizer.tokenize(x)))
#predict_list = list(test['comment'])
np_pre = np.asarray(predict_list)
print(np_pre[1])

['满', '满', '的', '五', '星', '好', '评', '，', '口', '味', '好', '，', '服', '务', '好', '，', '特', '别', '喜', '欢', '，', '昨', '天', '第', '一', '次', '买', '，', '今', '天', '就', '回', '购', '了', '，', '买', '的', '刨', '奶', '，', '店', '长', '问', '我', '加', '腰', '果', '还', '是', '核', '桃', '，', '我', '说', '随', '便', '，', '他', '又', '问', '我', '喜', '欢', '吃', '什', '么', '，', '我', '说', '都', '喜', '欢', '，', '然', '后', '，', '帅', '帅', '的', '店', '长', '都', '给', '我', '加', '了', '，', '超', '赞']


In [0]:
from kashgari import utils
processor = utils.load_processor(model_path='tf_bert_model/1')
tensor = processor.process_x_dataset(np_pre)
# tensor = [{
#    "Input-Token:0": i.tolist(),
#    "Input-Segment:0": np.zeros(i.shape).tolist()
# } for i in tensor]

In [0]:
print(tensor[1])

[   1  596  596    5  346  604  170  480    4  270  775  170    4  231
  112  170    4  169  348  692  811    4 2876  125  131    7  218 1042
    4  508  125  113  381  817   15    4 1042    5 4120 1618    4  737
   84  358   75  120 1844  228  201   10  592 1397    4   75  178  570
  518    4   44  311  358   75  692  811  943  614  356    4   75  178
  165  692  811    4  187   49    4 1822 1822    5  737   84  165  416
   75  120   15    4  634 1479    2    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [0]:
result = model.predict(tensor)
print(f"{result}")

UFuncTypeError: ignored